In [ ]:
# Fast small-MS write (metadata-only projection; no DATA rotation)
# - Keeps only the first N_TIMES_TO_KEEP timestamps to speed up IO
# - Recomputes UVW for a single ICRS/J2000 phase center
# - Writes to /tmp; change DEST_MS if you prefer a different location

import os
import sys
import time
import shutil
import numpy as np
import astropy.units as u
from pyuvdata import UVData

# ----- Config -----
SRC_ROOT = "/data/dsa110-contimg/src"
IN_DIR = "/data/incoming_test/2025-09-05T03-12-56_HDF5"
IN_FILE = os.path.join(
    IN_DIR, "2025-09-05T03:12:56_sb00.hdf5"
)  # single subband for speed
DEST_MS = "/tmp/quick_small.ms"  # change to your preferred path
N_TIMES_TO_KEEP = 2  # keep first 2 times for a quick run
WRITE_MODEL = False  # set True to write MODEL_DATA with a point source

# ----- Path bootstrap for imports in notebook -----
if SRC_ROOT not in sys.path:
    sys.path.insert(0, SRC_ROOT)

from dsa110_contimg.conversion.helpers import (
    set_model_column,
    phase_to_meridian,
)

# ----- Load UVH5 (single subband for speed) -----
t0 = time.time()
uv = UVData()
uv.read(
    IN_FILE,
    file_type="uvh5",
    run_check=False,
    run_check_acceptability=False,
    strict_uvw_antpos_check=False,
    check_extra=False,
)
uv.uvw_array = uv.uvw_array.astype(np.float64)

# ----- Optional: select first N_TIMES_TO_KEEP times to minimize size -----
unique_times = np.unique(uv.time_array)
if unique_times.size > N_TIMES_TO_KEEP:
    uv.select(times=unique_times[:N_TIMES_TO_KEEP], run_check=False)

# ----- Phasing -----
phase_to_meridian(uv)

print(f"Ntimes={uv.Ntimes}, Nbls={uv.Nbls}, Nblts={uv.Nblts}")
print("UVW finite fraction:", np.isfinite(uv.uvw_array).mean())

# ----- Write MS (fast path) -----
if os.path.exists(DEST_MS):
    shutil.rmtree(DEST_MS)

uv.write_ms(
    DEST_MS,
    clobber=True,
    run_check=False,
    check_extra=False,
    run_check_acceptability=False,
    strict_uvw_antpos_check=False,
    check_autos=False,
    fix_autos=False,
)
print(f"Wrote MS: {DEST_MS} in {time.time() - t0:.1f}s")

# ----- Optional: write MODEL_DATA for a point source at phase center -----
if WRITE_MODEL:
    pt_dec = uv.extra_keywords.get("phase_center_dec", 0.0) * u.rad
    ra_icrs = uv.phase_center_ra * u.rad
    dec_icrs = uv.phase_center_dec * u.rad
    set_model_column(DEST_MS[:-3], uv, pt_dec, ra_icrs, dec_icrs, flux_Jy=10.0)
    print("MODEL_DATA written.")

2025-10-07 17:49:03 [INFO] Setting DSA-110 antenna positions
2025-10-07 17:49:04 [INFO] Loaded dynamic antenna positions for 117 antennas


Ntimes=2, Nbls=4656, Nblts=9312
UVW finite fraction: 1.0
Wrote MS: /tmp/quick_small.ms in 12.9s


listobs written to: /tmp/quick_small.listobs
Successful readonly open of default-locked table /tmp/quick_small.ms::FIELD: 9 columns, 1 rows
FIELDS: ['FIELD_CENTER'] (total: 1 )
PHASE_DIR shape: (1, 1, 2)
Successful readonly open of default-locked table /tmp/quick_small.ms::SPECTRAL_WINDOW: 16 columns, 1 rows
SPW: nchan = 48 freq ascending: True
SPW: channel_width positive: True
Successful readonly open of default-locked table /tmp/quick_small.ms: 23 columns, 9312 rows
UVW rows: 9312 min(m): -1836.1816634101265 max(m): 1695.5881506802814
DATA sample shape (npol, nchan, nrow=1): (1, 48, 2)
Successful readonly open of default-locked table /tmp/quick_small.ms: 23 columns, 9312 rows
Has MODEL_DATA: False Has CORRECTED_DATA: False


2025-10-07 17:28:52 [INFO] Setting DSA-110 antenna positions
2025-10-07 17:28:53 [INFO] Loaded dynamic antenna positions for 117 antennas


Nblts, Ntimes, Nbls: 111744 24 4656
UVW finite fraction: 1.0
Unique frames in catalog: {'icrs'}


KeyboardInterrupt: 

NameError: name 'uv' is not defined